<a href="https://colab.research.google.com/github/hmaldonado12/7506R-1C2024-GRUPO6/blob/main/tp2/XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Primero cargo los csv:

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import nltk

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv/train.csv')
train_df.head()

,ID,review_es,sentimiento
0,0,Uno de los otros críticos ha mencionado que de...,positivo
1,1,Una pequeña pequeña producción.La técnica de f...,positivo
2,2,Pensé que esta era una manera maravillosa de p...,positivo
3,3,"Básicamente, hay una familia donde un niño peq...",negativo
4,4,"El ""amor en el tiempo"" de Petter Mattei es una...",positivo


In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test.csv/test.csv')
test_df.head()

,ID,review_es
0,60000,La mayor virtud de esta película es su existen...
1,60001,"No soy un experto cinéfilo, pero pocas veces m..."
2,60002,Si no eres un incondicional del humor estilo T...
3,60003,"No sé qué está pasando, si la gente se deja ll..."
4,60004,"Pero cuando amanece,y me quedo solo,siento en ..."


Para el preprocesamiento de los datos hay que limpiar y preprocesar los textos para que los modelos puedan manejarlos adecuadamente.

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def preprocess_text(text):
    # tokenizacion
    tokens = word_tokenize(text.lower())
    # elimino signos de puntuación
    tokens = [word for word in tokens if word not in string.punctuation]
    # elimino stopwords
    stop_words = set(stopwords.words('spanish'))
    tokens = [word for word in tokens if word not in stop_words]
    return " ".join(tokens)

train_df['review_es'] = train_df['review_es'].apply(preprocess_text)
test_df['review_es'] = test_df['review_es'].apply(preprocess_text)

In [ ]:
train_df.head()

,ID,review_es,sentimiento
0,0,críticos mencionado después ver solo 1 oz epis...,positivo
1,1,pequeña pequeña producción.la técnica filmació...,positivo
2,2,pensé manera maravillosa pasar tiempo fin sema...,positivo
3,3,básicamente familia niño pequeño jake piensa z...,negativo
4,4,`` amor tiempo '' petter mattei película visua...,positivo


In [ ]:
test_df.head()

,ID,review_es
0,60000,mayor virtud película existencia.el hecho poda...
1,60001,experto cinéfilo pocas veces tan juego sala ci...
2,60002,si incondicional humor estilo tele 5.si termin...
3,60003,sé pasando si gente deja llevar modas si verda...
4,60004,amanece quedo solo siento fondoun mar vacío se...


Feature Enconding:

Uso Bag of Words (BoW) para convertir el texto en vectores numéricos que puedan ser utilizados por los modelos de machine learning.

In [ ]:
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(train_df['review_es'])
y_train = train_df['sentimiento']

In [ ]:
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

//////////////////////////////////////////////////////////////////////

# XGBoost

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder

In [ ]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'eta': 0.1,
    'max_depth': 6,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'verbosity': 0
}

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train_split)
y_val_encoded = label_encoder.transform(y_val_split)

dtrain = xgb.DMatrix(X_train_split, label=y_train_encoded)
dval = xgb.DMatrix(X_val_split, label=y_val_encoded)
model = xgb.train(params, dtrain, num_boost_round=1000, evals=[(dval, 'validation')], early_stopping_rounds=10, verbose_eval=False)


Evaluación del modelo:

In [ ]:
y_pred = model.predict(dval)
y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]

Metricas:

In [ ]:
y_val_pred_labels = label_encoder.inverse_transform(y_pred_binary)
y_val_true_labels = label_encoder.inverse_transform(y_val_encoded)

# Imprimir métricas de evaluación
print("Accuracy:", accuracy_score(y_val_true_labels, y_val_pred_labels))
print("Precision:", precision_score(y_val_true_labels, y_val_pred_labels, pos_label='positivo'))
print("Recall:", recall_score(y_val_true_labels, y_val_pred_labels, pos_label='positivo'))
print("F1 Score:", f1_score(y_val_true_labels, y_val_pred_labels, pos_label='positivo'))

print("Classification Report:\n", classification_report(y_val_true_labels, y_val_pred_labels))


Accuracy: 0.8723
Precision: 0.8635485117897178
Recall: 0.8866838658463981
F1 Score: 0.874963282091452
Classification Report:
               precision    recall  f1-score   support

    negativo       0.88      0.86      0.87      4961
    positivo       0.86      0.89      0.87      5039

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000



Realizar predicciones en el conjunto de prueba (test.csv)

In [ ]:
X_test = vectorizer.transform(test_df['review_es'])
dtest = xgb.DMatrix(X_test)
test_predictions = model.predict(dtest)


Se decodifica las predicciones a etiquetas originales

In [ ]:
test_predictions_labels = label_encoder.inverse_transform([1 if pred > 0.5 else 0 for pred in test_predictions])

Ahora se prepara el archivo csv para subrilo a kaggle

#Exportar CSV para Submission

In [ ]:
# Creo nuevo DF con ID de reseña y predicción de sentimiento
output_df = pd.DataFrame({'ID': test_df['ID'], 'sentimiento': test_predictions_labels})
output_df

,ID,sentimiento
0,60000,negativo
1,60001,positivo
2,60002,negativo
3,60003,negativo
4,60004,negativo
...,...,...
8594,68594,positivo
8595,68595,negativo
8596,68596,positivo
8597,68597,negativo


In [ ]:
# Genero archivo CSV
output_df.to_csv('/content/drive/My Drive/Colab Notebooks/XGBoost.csv', index=False)